In [19]:
from pandasql import sqldf

In [20]:
mysql=lambda q: sqldf(q,globals())

# 테이블의 구조 및 데이터

# 릴레이션이 아닌 예시

# 릴레이션의 구조

# 외래키의 예제

# SQL의 문장 구조

## OR조건의 검색

## AND조건의 검색

In [9]:
SELECT 도서명, 판매가 
FROM 도서
WHERE 저자명='이춘열'
AND 저자명='김인재';

SyntaxError: invalid syntax (169773306.py, line 1)

In [10]:
#INTERSECT로 교집합 구하기(Mysql은 intersect지원안함)
(SELECT 도서명, 판매가
FROM 도서
WHERE 저자명='이춘열')

INTERSECT

(SELECT 도서명, 판매가
FROM 도서
WHERE 저자명='김인재')

SyntaxError: invalid syntax (2593699385.py, line 2)

In [ ]:
#대신사용가능 방법_서브쿼리 이용
SELECT 도서명, 판매가
FROM 도서
WHERE 저자명='이춘열'
AND 도서명 in(SELECT 도서명
              FROM 도서
              WHERE 저자명='김인재')

# SQL명령어들

## Join

In [ ]:
- (INNTER)JOIN
- OUTER JOIN

### Join 예시

In [22]:
import pandas as pd
menu_df=pd.read_csv('Downloads/menu.txt')
menu_df

,id,name,price,kcal
0,D0,Chocolate Parfeit,4900,420
1,D1,Pudding Parfeit,5300,380
2,D2,Strawberry Parfeit,5200,380
3,D3,Panna Cotta,4200,283
4,D4,Cheese Mousse,5800,288
5,D5,Affogato,3000,248
6,D6,Tiramisu,6000,251
7,D7,Greentea Parfeit,4500,380
8,D8,Vanilla Gelato,3600,131
9,D9,Caramel Pancake,3900,440


In [23]:
order_df=pd.read_csv('Downloads/orders.txt',header=None,names=['id','menuid','qty'] )
order_df

,id,menuid,qty
0,S1,D0,2
1,S1,D1,1
2,S1,D8,2
3,S2,D10,4
4,S2,D5,1
5,S3,D8,2
6,S3,D2,1
7,S4,D5,3
8,S5,D8,2


### OUTER JOIN

In [31]:
#조인 조건을 만족시키지 않는 레코드들도 표시
# LEFT(OUTER)JOIN, RIGHT (OUTER)JOIN
mysql("""select menu_df.*,order_df.*
        from menu_df left join order_df
        on(menu_df.id=order_df.menuid)""")

,id,name,price,kcal,id,menuid,qty
0,D0,Chocolate Parfeit,4900,420,S1,D0,2.0
1,D1,Pudding Parfeit,5300,380,S1,D1,1.0
2,D2,Strawberry Parfeit,5200,380,S3,D2,1.0
3,D3,Panna Cotta,4200,283,None,None,NaN
4,D4,Cheese Mousse,5800,288,None,None,NaN
5,D5,Affogato,3000,248,S2,D5,1.0
6,D5,Affogato,3000,248,S4,D5,3.0
7,D6,Tiramisu,6000,251,None,None,NaN
8,D7,Greentea Parfeit,4500,380,None,None,NaN
9,D8,Vanilla Gelato,3600,131,S1,D8,2.0


## 집합관련 명령어들

In [ ]:
- UNION(합집합)
- INTERSECT(교집합):어떤 SQL은 INTERSECT명령어 지원안함
- MINUS(차집합): 어떤 SQL은 MINUS명령어 지원안함

## 서브쿼리

In [ ]:
# 다른 쿼리에 포함된 SQL쿼리
# 중첩쿼리
# ()안에만 표시

In [37]:
#값이 평균보다 싼 메뉴들은?
mysql("""select id, name, price
        from menu_df
        where price<(select avg(price)
                    from menu_df)""")

,id,name,price
0,D3,Panna Cotta,4200
1,D5,Affogato,3000
2,D7,Greentea Parfeit,4500
3,D8,Vanilla Gelato,3600
4,D9,Caramel Pancake,3900
5,D10,Apple Pie,4400


## GROUP BY

In [25]:
#order_df와 menu_df로부터, 메뉴별 판매 금액 계산하고자 함
#1. 주문을 메뉴id별로 모아서(groupby명령어)
#2. 주문 수량과 메뉴 가격을 곱하고
#3. 곱한 금액을 다 합하여 계산

In [27]:
sales_menu=mysql("""select menu_df.id as menuid,menu_df.name as name,
                        sum(order_df.qty*menu_df.price) as sales_money
                    from order_df join menu_df
                    on(order_df.menuid=menu_df.id)
                    group by menu_df.id""")
sales_menu

,menuid,name,sales_money
0,D0,Chocolate Parfeit,9800
1,D1,Pudding Parfeit,5300
2,D10,Apple Pie,17600
3,D2,Strawberry Parfeit,5200
4,D5,Affogato,12000
5,D8,Vanilla Gelato,21600


### Having

In [28]:
#order_df와 menu_df로부터 판매액이 10000원 이상인 메뉴 찾고자함
#1. 주문을 메뉴id별로 모아서(groupby명렁어)
#2. 주문 수량과 메뉴가격을 곱하고
#3. 곱한 금액을 다 합한 후
#4. 합계가 10000원이상인 메뉴를 검색(having명령어)


In [29]:
sales_menu_good=mysql("""select menu_df.id as menuid,menu_df.name as name,
                        sum(order_df.qty*menu_df.price) as sales_money
                    from order_df join menu_df
                    on(order_df.menuid=menu_df.id)
                    group by menu_df.id
                    having sales_money>=10000""")
sales_menu_good

,menuid,name,sales_money
0,D10,Apple Pie,17600
1,D5,Affogato,12000
2,D8,Vanilla Gelato,21600


In [35]:
#잘 안팔리는 메뉴들: 메뉴별 판매수량이 적은(<2) 메뉴들
#1. 전체 판매량이 <2, 즉 2개도 안팔린 메뉴
#판매량이 0인 메뉴는 어디로?-> null을 0으로 자동변환해주지 않음

In [36]:
mysql("""select menu_df.*,sum(order_df.qty) as orderTotal
        from menu_df left join order_df
        on (menu_df.id=order_df.menuid)
        group by menu_df.id
        having orderTotal<2""")

,id,name,price,kcal,orderTotal
0,D1,Pudding Parfeit,5300,380,1
1,D2,Strawberry Parfeit,5200,380,1


## IS NULL

In [34]:
#주문이 없는 메뉴
# OUTER JOIN한 칼럼의 값이 NULL인 레코드들
mysql("""select menu_df.*
        from menu_df left join order_df
        on(menu_df.id=order_df.menuid)
        where order_df.menuid is null""")

,id,name,price,kcal
0,D3,Panna Cotta,4200,283
1,D4,Cheese Mousse,5800,288
2,D6,Tiramisu,6000,251
3,D7,Greentea Parfeit,4500,380
4,D9,Caramel Pancake,3900,440
